# Multi Search RAG Agent

**Tools**

Tools are utilities designed to be called by a model: their inputs are designed to be generated by models, and their outputs are designed to be passed back to models.

A toolkit is a collection of tools meant to be used together.

In [1]:
!pip install arxiv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# To import WikipediaQueryRun for querying Wikipedia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=4, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="llama2"))
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E7FF2D56A0>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(
    retriever, 
    "langsmith_search",
    "Search for information about LangSmith. For any questions about Langsmith, you must use this tool.")

In [8]:
retrieval_tool.name

'langsmith_search'

In [9]:
# ArXiv Query Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=4, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [12]:
tools = [wiki, arxiv, retrieval_tool] # List of tools to be used in the agent in order of how they will be called/searched

In [13]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\revna\\Desktop\\Projects\\Multi-Search-RAG-Agent\\RAG_venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=4, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=4, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about Langsmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E81F6DC2C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'

# Agents

Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. 

In [14]:
from dotenv import load_dotenv
load_dotenv()
import os

In [28]:
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatOllama
from langchain.agents import AgentExecutor
from langchain.prompts import PromptTemplate

llm = ChatOllama(model="llama2", temperature=0)

# ReAct prompt template - this is the standard format
react_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")

# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True  # This helps with local model parsing issues
)


In [31]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Great, let's get started! Here's my response:

Action: langsmith_search
Action Input: query: 'LangSmith', retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E7FF2D56A0>, search_kwargs={}), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator: 'str' = '\n\n', callbacks: 'Callbacks' = None, response_format: "Literal['content', 'content_and_artifact']" = 'content')
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performan

KeyboardInterrupt: 

In [32]:
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatOllama
from langchain.agents import AgentExecutor
from langchain.prompts import PromptTemplate

llm = ChatOllama(model="llama2", temperature=0)
tools = [wiki, arxiv]

# ReAct prompt template - this is the standard format
react_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")

# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True  # This helps with local model parsing issues
)


In [33]:
agent_executor.invoke({"input":"What is AI?"})



> Entering new AgentExecutor chain...
Question: What is AI?
Thought: Hmm, that's a broad question. AI stands for Artificial Intelligence, which refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.
Action: arxiv
Action Input: search query - "AI definition"No good Arxiv Result was foundQuestion: What is AI?
Thought: Hmm, that's a broad question. AI stands for Artificial Intelligence, which refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.
Action: wikipedia
Action Input: search query - "AI definition"Page: GPT-4
Summary: Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model trained and created by OpenAI and the fourth in its series of GPT foundation models. It was launched on March 14, 2023, and made publicly availableQuestion: Wh

{'input': 'What is AI?',
 'output': 'Agent stopped due to iteration limit or time limit.'}